<a href="https://colab.research.google.com/github/miinzi/movie_review_2021/blob/main/Naver_Movie_Review_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naver Movie Review Crawler(2021.12.19 배포)

아래 코드는 논문 'KoBERT 다중분류 모델을 이용한 감정분석 기반 영화 추천 시스템 연구'에서 사용되었습니다.

*A study on the Movie Recommendation System based on Emotional Analysis using the KoBERT Multi-classification Model*

 > 코드 작성자 (2021.12.19) 김민지 *https://github.com/miinzi/movie_review_2021*
 
 > 참조 코드 (2021.04.21) 박경태 *https://ktae23.tistory.com/*

### 필요한 라이브러리를 불러옵니다



In [ ]:
import requests 
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import openpyxl
from urllib.request import urlretrieve
import ssl
import re
import os
from random import randrange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 사용할 함수를 정의합니다

In [ ]:
CRAWLING_DATE = "20211120" ### 랭킹 기준 날짜
movie_index = 0 ### 전체 영화 수
review_index = 0 ### 전체 리뷰 링크

In [ ]:
movie_dataset = pd.DataFrame(columns=['movie_index', 'movie_code',	'neutral', 'fear',	'surprise',	'angry',	'sad',	'happy',	'aversion',	'movie_title',	'movie_genre',	'movie_director', 'movie_actor',	'movie_story',	'movie_date',	'review_num',	'movie_link',	'date',	'moive_time',	'movie_rate'])
review_dataset = pd.DataFrame(columns = ['review_index', 'movie_code', 'score', 'content'])

In [ ]:
movie_dataset

,movie_index,movie_code,neutral,fear,surprise,angry,sad,happy,aversion,movie_title,movie_genre,movie_director,movie_actor,movie_story,movie_date,review_num,movie_link,date,moive_time,movie_rate


In [ ]:
review_dataset

,review_index,movie_code,score,content


In [ ]:
# k번째 웹페이지에서 소스코드를 가져와 BeautifulSoup으로 파싱
def movie_page_crawling(k):
    raw = requests.get("https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date="+CRAWLING_DATE+"&page="+str(k), headers={'User-Agent':'Mozilla/5.0'})
    html = BeautifulSoup(raw.text, 'html.parser')
    movie_crawling(html)

In [ ]:
# 특수문자 제거를 위한 함수
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
    return text

In [ ]:
# 페이지 내 50개 영화에 대한 크롤러
def movie_crawling(html):
    global movie_index
    global review_index
    global movie_dataset
    global review_dataset

    movie = html.select("td.title")
    # print(movie)

    for m in movie:
        m_title = m.select_one("a")
        url = "https://movie.naver.com" + m_title.attrs["href"]
        # print(url)
        
        movie_code = url[53:] ### 영화 고유코드 추출

        # url(상세페이지)에 접속, html 파싱
        raw_each = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
        html_each = BeautifulSoup(raw_each.text, 'html.parser')
        
        # (1) 전체 컨테이너
        movie_one = html_each.select("div.article")

        # (2) 전체 컨테이너가 갖고 있는 영화관련 정보
        for a, m_o in enumerate(movie_one):

            # (3-1) 영화제목 수집
            title = m_o.select_one("h3.h_movie a")

            # (3-2) 영화평점 수집
            score = m_o.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")

            # (3-3) 영화장르 수집
            genre = m_o.select("dl.info_spec dd p span:nth-of-type(1) a")

            # (3-4) 영화감독 수집
            directors = m_o.select("dl.info_spec dd:nth-of-type(2) a")

            # (3-5) 영화배우 수집
            actors = m_o.select("dl.info_spec dd p:nth-of-type(3) a")

            # (3-6) 영화줄거리 수집
            story = m_o.select("div.story_area p.con_tx")

            # (3-7) 영화 개봉일 수집
            rdate = m_o.select("dl.info_spec dd p span:nth-of-type(4):nth-child(n+3):nth-child(-n+4)")

            # (3-8) 영화 상영시간 수집
            rtime = m_o.select_one("dl.info_spec dd p span:nth-of-type(3)")

            # 평점 개수 수집
            num_review = m_o.select(".score_total strong.total em")[1].get_text().strip().replace(",", "")
            # print(num_review)

            # (4) skip 처리-1: 평점이 없으면 넘어간다.
            non_score = "관람객 평점 없음"
            if (score == None) or (non_score in score.text):
                is_ok = False
                continue

            # (4-1) score에서 점수 부분만 남기고 숫자로 변환한다.
            score = score.text
            score = score[6:11]


            # (5) skip 처리 : 주연배우에 관람 기준이 적혀있거나 배우가 없을 경우 넘어간다.
            if len(actors) > 1:
                if "청소년 관람불가" in actors[0].text:
                    is_ok = False
                    continue
                elif "12세 관람가" in actors[0].text:
                    is_ok = False
                    continue
                elif "전체 관람가" in actors[0].text:
                    is_ok = False
                    continue

            elif len(actors) == 0:
                continue

            # (7-1) 데이터 만들기-1 : HTML로 가져온 영화장르/영화감독/영화배우 정보에서 TEXT정보만 뽑아서 리스트 형태로 만들기
            genre_list = [g.text for g in genre]
            directors_list = [d.text for d in directors]
            actors_list = [a.text for a in actors]
            story_list = [s.text for s in story]
            rdate_list = [r.text for r in rdate]


            # (7-2) 데이터 만들기-2 : 여러 개로 이루어진 리스트 형태를 하나의 문자열 형태로 만들기
            genre_str = ','.join(genre_list)
            directors_str = ','.join(directors_list)
            actors_str = ','.join(actors_list)
            story_str = ','.join(story_list)
            rdate_str = ','.join(rdate_list)
            index = rdate_str.rfind('개')
            rdate_year = rdate_str[index-12:index-8]

            # 개봉일에서 모든 공백 제거
            rdate_str = re.sub('[\s]', '', rdate_str) 

            print("=" * 50)
            print("영화코드:", movie_code)        

            print("=" * 50)
            print("제목:", title.text)
            
            print("=" * 50)
            if score != None:
                print("평점:", score)

            print("=" * 50)
            print("장르:")
            for g in genre:
                print(g.text)

            print("=" * 50)
            print("감독:")
            for d in directors:
                print(d.text)

            print("=" * 50)
            print("주연 배우:")
            for a in actors:
                print(a.text)

            print("=" * 50)
            print("줄거리:")
            for s in story:
                print(s.text)

            print("=" * 50)
            print("개봉일:")
            for rd in rdate:
                rd = rd.text
                rd = re.sub('[\s]', '', rd)
                print(rd)

            print("=" * 50)
            print("상영시간:")
            print(rtime.text)

            print("=" * 50)
            print("개봉년도:")
            print(rdate_year)

            print("=" * 50)

            new_movie = {
                'movie_index': movie_index,
                'movie_code': movie_code,
                'neutral': 0,
                'fear': 0,
                'surprise': 0,
                'angry': 0,
                'sad': 0,
                'happy': 0,
                'aversion': 0,
                'movie_title': title.text,
                'movie_genre': genre_str,
                'movie_director': directors_str,
                'movie_actor': actors_str,
                'movie_story': story_str,
                'movie_date': rdate_str,
                'review_num': num_review,
                'movie_link': url,
              	'date': rdate_year,
                'moive_time': rtime.text,
                'movie_rate': score
            }

            movie_dataset = movie_dataset.append(new_movie,ignore_index=True)
            movie_index += 1

            # 평점리뷰 페이지 이동
            page_num = int(num_review[:len(num_review)-1]) + 1
            print(page_num)

            for p in range(1, page_num+1):
                url_point = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code="+movie_code+"&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page="+str(p)
                raw_point = requests.get(url_point, headers={"User-Agent":"Mozilla/5.0"})
                html_point = bs(raw_point.text, 'html.parser')
                # print(html_point)

                # score_num = html_point.select("body > div > div > div.score_total > strong > em")
                # print(+ str(score_num))

                reply = html_point.select("div.score_result li")
                print("-" * 50)
                print("평점과 댓글("+str(p)+"):")
                for r in reply:

                    score_each = r.select_one("div.star_score em").text
                    # reply = r.select_one("div.score_reple p").text[4:].lstrip()

                    # 예외처리
                    reply_each = r.select_one("div.score_reple p").text.lstrip()
                    if reply_each[:17] == "스포일러가 포함된 감상평입니다.":
                      reply_each = reply_each[25:].lstrip().rstrip()
                    if reply_each[:3] == "관람객":
                      reply_each = reply_each[3:].lstrip().rstrip()
                    # reply_each = cleanText(reply_each)

                    new_review = {
                        'review_index': review_index,
                        'movie_code': movie_code,
                        'score': score_each,
                        'content': reply_each
                    }
                    
                    review_dataset = review_dataset.append(new_review,ignore_index=True)
                    print(score_each, reply_each)
                    review_index += 1

### 정보를 수집 후 파일로 저장합니다

In [ ]:
# !pip install xlsxwriter

In [ ]:
# for i in range(1, 4):
#   movie_page_crawling(i)

movie_page_crawling(1)
movie_dataset.set_index('movie_index', inplace=True)
review_dataset.set_index('review_index', inplace=True)

# movie_dataset.to_excel("/content/drive/#구글드라이브 내 파일 저장위치#/movie_data("+CRAWLING_DATE+")_51_60.xlsx", engine='xlsxwriter')
# review_dataset.to_excel("/content/drive/#구글드라이브 내 파일 저장위치#/review_data("+CRAWLING_DATE+")_51_60.xlsx", engine='xlsxwriter')

movie_dataset.to_csv("/content/drive/#구글드라이브 내 파일 저장위치#/movie_data("+CRAWLING_DATE+")_200_250.csv", index=True, encoding='utf-8-sig')
review_dataset.to_csv("/content/drive/#구글드라이브 내 파일 저장위치#/review_data("+CRAWLING_DATE+")_200_250.csv", index=True, encoding='utf-8-sig')

영화코드: 186114
제목: 밥정
평점:  9.70
장르:
다큐멘터리
드라마
감독:
박혜령
주연 배우:
임지호
줄거리:
잔디, 잡초, 이끼, 나뭇가지.. 자연을 재료 삼아 요리를 만드는 방랑식객 임지호 셰프.  친어머니와 양어머니에 대한 아픈 사연을 간직한 그는 길에서 인연을 맺은 사람들에게 기꺼이 음식을 대접한다. 지리산에서 만난 김순규 할머니를 길 위의 어머니로 10년간 모시지만, 끝끝내 찾아온 3번째 이별. 임지호 셰프는 낳아주신, 길러주신, 그리고 마음을 나눠주신 3명의 어머니를 위해 3일 동안 108접시의 음식을 장만한다. ‘밥’으로 ‘정’을 나누는 인생의 참맛, 더 늦기 전에 당신과 나누고 싶습니다…
개봉일:
2020.10.07개봉
상영시간:
82분 
개봉년도:
2020
36
--------------------------------------------------
평점과 댓글(1):
--------------------------------------------------
평점과 댓글(2):
--------------------------------------------------
평점과 댓글(3):
--------------------------------------------------
평점과 댓글(4):
--------------------------------------------------
평점과 댓글(5):
--------------------------------------------------
평점과 댓글(6):
--------------------------------------------------
평점과 댓글(7):
--------------------------------------------------
평점과 댓글(8):
--------------------------------------------------
평점과 댓글(9):
--------------------------------------------------
평

KeyboardInterrupt: ignored